# DS GA 1004 Capstone Project Team 27
## Giulio Duregon, Joby George, Jonah Poczobutt 

## Local Playground on Small Dataset

0. Imports
1. Read the data (locally, for now)
2. Clean the data (UTF-8 encoding)
3. Create Train/Test/Validation
4. Create a baseline model


Note I have some cells below that will be useful if using the HPC rather than downloading the data locally


In [4]:
#0 Imports 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

small_file_path = 'work/courses/DSGA1004-2021/movielens/ml-latest-small/'

In [ ]:
#pwd

you'll need to play with the number of cd ../ until you get to the **scratch** directory, on the HPC so you can read the data, for me it was the below:



In [ ]:
#cd ../..

In [6]:
#1 Read the data


movies = pd.read_csv(small_file_path+'movies.csv')
ratings = pd.read_csv(small_file_path + 'ratings.csv')
tags = pd.read_csv(small_file_path + 'tags.csv')
links = pd.read_csv(small_file_path + 'links.csv')

# commented code when working on local running, non commented code is on the HPC
# Data are organized in links.csv, movies.csv, ratings.csv, and tags.csv
# path = 'ml-latest-small/'

# movies = pd.read_csv(path+'movies.csv')
# ratings = pd.read_csv(path + 'ratings.csv')
# tags = pd.read_csv(path + 'tags.csv')
# links = pd.read_csv(path + 'links.csv')





now cd back to my directory

In [ ]:
cd jg6615/capstone_project

In [4]:
#Clean the data
def clean_dates(df,column):
    """
    df = dataframe with a column we want to convert from utc time data to d/m/Y
    column - string of column name, 
    """
    df[column] = pd.to_datetime(df[column]).dt.date
    return(df)

#we don't actually have to clean the timestamps for the baseline model


#ratings = clean_dates(ratings,'timestamp')
#tags = clean_dates(tags,'timestamp')



def seperate_genres(df,column):
    """
    df = dataframe with a column we want to convert from utc time data to d/m/Y
    column - string of column name, 
    """
    genre_cols = df[column].str.get_dummies()
    df = df.drop(columns = [column])
    df = df.join(genre_cols, how='left', on=df.index)
    return(df)

#we don't have to clean genres for the baseline model, either

#movies = seperate_genres(movies, 'genres')



In [7]:
#some titles have more than one movieId, 
#I wanted my sparsely populated matrix to be uesrs as rows
#movie title on columns, so to correct for this

#we identify titles with the more than 1 id, and remove the ids from the dataset with the least reviews
#for a given title

#find movies with multiple movie Ids
g = movies.groupby('title').movieId.count()>1
dupes = list(movies.groupby('title').movieId.count()[g].index)
print('titles with more than one movieId in the movies data ',dupes)
d = {title:movies.loc[movies.title == title]['movieId'].values for title in dupes}



titles with more than one movieId in the movies data  ['Confessions of a Dangerous Mind (2002)', 'Emma (1996)', 'Eros (2004)', 'Saturn 3 (1980)', 'War of the Worlds (2005)']


In [8]:
#examine for the same title, which movie id has the fewest amount of reviews
#we will remove those movies from the dataset such that every movie title has a singular movieId
discard_ids = []
for v in d.values():
    g = ratings.loc[ratings['movieId'].isin(v)].groupby('movieId').count()
    #keep only the movie id with the most amount of reviews for each title with multiple
    #movieIds
    discard_ids.append(g.sort_values(by='userId',ascending=True).iloc[-1].name)
    
#remove movieId's with a duplicate title    
movies = movies.loc[~(movies.movieId.isin(discard_ids))]
ratings = ratings.loc[~(ratings.movieId.isin(discard_ids))]

#join ratings and movie data together
utility_matrix = ratings.merge(movies, how = 'left', on = 'movieId')

#subset on columns
u = utility_matrix[['userId','title','rating']]

#create sparse matrix transforming long data into wide with the .pivot() method
u = u.pivot(index='userId', columns = 'title', values ='rating')

In [9]:
u.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#hard part -- create a train val test split!
#from lecture we want to make sure we subset a portion of each user into the val and test split!
# meaning 70% of records from user 1 is in the training set, and 30% are further hold out

#strategy, initialize 3 empty dataframes in the same dimmensions as our sparse matrix
fill = u.shape
train= pd.DataFrame(np.zeros(fill), columns = u.columns)
val = pd.DataFrame(np.zeros(fill), columns = u.columns)
test = pd.DataFrame(np.zeros(fill), columns = u.columns)


#takes a couple of minutes for this to run, could be worth using dask or something 
#iterate from 0 -609, we reset index at the end to get our row num to equal the previous userId

for idx in range(len(u.index)):
    #get a movie name, rating for all movies that this user has rated, from the sparsely populated matrix
    #note that u matrix has NANs instead of 0s so the notnull method grabs what we want
    first_guess = u.iloc[idx].loc[u.iloc[idx].notnull()]
    
    
    #get 60 percent of the data to split into train, 40 % to split into test, which will be further split
    #into validation
    msk1 = np.random.rand(len(first_guess)) < 0.6
    train_cols, big_test_cols = list(first_guess[msk1].index), list(first_guess[~msk1].index)
    train_vals, big_test_vals = list(first_guess[msk1].values),list(first_guess[~msk1].values)
    
    #create a dictionary that will be passed into the .replace() method to assign the true ratings for this 
    #user for the given set of movies in train and big test 
    d_train = {train_cols[i]:{0:train_vals[i]} for i in range(len(train_cols))}

    #modify the row to replace 0s with the appropriate value, using replace
    train.iloc[idx] = train.replace(d_train).iloc[idx]

    #subset big_test into true validation/test sets, taking 50% of 40% (i.e. 20%)
    msk2 = np.random.rand(len(big_test_cols)) < 0.5
    val_cols, test_cols = list(first_guess[~msk1][msk2].index), list(first_guess[~msk1][~msk2].index)
    val_vals, test_vals = list(first_guess[~msk1][msk2].values),list(first_guess[~msk1][~msk2].values)
    
    d_val = {val_cols[i]:{0:val_cols[i]} for i in range(len(val_cols))}
    d_test = {test_cols[i]:{0:test_vals[i]} for i in range(len(test_cols))}

    #update the validation and test datasets to be a matrix in the same dimmension as u, but with
    #that given user's rating for each movie 
    val.iloc[idx] = val.replace(d_val).iloc[idx]
    test.iloc[idx] = test.replace(d_test).iloc[idx]

#replace 0's with NaN's
train = train.replace(0, np.nan)
val = val.replace(0, np.nan)
test = test.replace(0, np.nan)

#set indicidees to match the original, my indexing was off by 1 (no userId = 0)
train = train.set_index(u.index)
val = val.set_index(u.index)
test = test.set_index(u.index)

IndexError: single positional indexer is out-of-bounds

In [23]:
train.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
#validate that for a row, the data in the train, val and test split are exactly 
#the same as the original dataset
a, b = train.iloc[26].loc[train.iloc[26].notnull()], val.iloc[26].loc[val.iloc[26].notnull()]
c = test.iloc[26].loc[test.iloc[26].notnull()]
d= list(a.index)+list(b.index)+list(c.index)
e = u.iloc[27].loc[u.iloc[27].notnull()]

f = list(e.index.sort_values())
##commented code will work on the next time i run it, i made a mistake in indexing above in my loop
# a, b = train.iloc[27].loc[train.iloc[27].notnull()], val.iloc[27].loc[val.iloc[27].notnull()]
# c = test.iloc[27].loc[test.iloc[27].notnull()]
# d= list(a.index)+list(b.index)+list(c.index)
# e = u.iloc[27].loc[u.iloc[27].notnull()]

In [92]:
#confirm list for userId 28, movies they've rated are the same 
d.sort()
d == f

True

In [32]:
e['Wrestler, The (2008)']

3.0

In [93]:
## confirming that ratings are the same  for a single movie 
e['Wrestler, The (2008)'] == c['Wrestler, The (2008)'] #move is exclusively in testing data

#a['Wrestler, The (2008)'] not present in training data
#b['Wrestler, The (2008)'] not present in validation data

True

In [94]:
#lastly, making sure the proportions are correct (60% train, 20% val, 20% test)
print(len(e)*.6, len(a),len(b),len(c))
352+107+110 == len(e) 
#percentages are rough, but it checks out, would have expected ~113 in test/val but this is good enough 

341.4 350 110 109


True

## Maybe Useful, later

In [ ]:

#u =  u.pivot(index='userId', columns = 'title', values ='rating')
#https://towardsdatascience.com/working-with-sparse-data-sets-in-pandas-and-sklearn-d26c1cfbe067 
#make a sparse dataframe, because we can use sci-py and sci-kit learn to get our train test validation split
#light fm doesn't work on greene
#this also one hot encodes userId and movie, which is useful for when we do splitting on X,y
# sparse_u = pd.get_dummies(u, columns=['userId', 'title'], sparse=True)

# y_sparse = sparse_u['rating']
# X_sparse = scipy.sparse.csr_matrix(sparse_u[sparse_u.columns.difference(['rating'])])





In [ ]:


#def get_release_year(df,column):
    """
    df = dataframe with a column we want to convert from utc time data to d/m/Y
    column - string of column name, 
    """
    #grabs the last 4 characters, usually a year, but does not work all the time for all movies on small dataset
    #release_year_array = df[column].str[-5:-1]
    #df.Salary.apply(lambda x: np.where(x.isdigit(),x,'0'))
    #will need to see how the full dataset has missing years, and how to do deal with movies with no year
    #return(df)